
# Klasyfikacja obrazów z użyciem sieci konwolucyjnej (CNN) w TensorFlow

**Autor:** Marcin Albiniak (EXE.EDU)  
**Opis:** Kompletny pipeline: przygotowanie danych, budowa modelu CNN, trening, zapis wag, wizualizacja efektów klasyfikacji, predykcja na nieznanych obrazach.  
**Zbiór danych:** MNIST (można łatwo zmienić na CIFAR-10/własny).

**Wymagania:**  
`tensorflow`, `matplotlib`, `numpy`, `scikit-learn`, `seaborn`


In [ ]:

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns


## 1. Przygotowanie danych

In [ ]:

# Wczytujemy przykładowy zbiór MNIST (obrazy cyfr 28x28, 10 klas)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalizacja do [0,1]
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# Podział na train/val/test
val_size = 5000
x_val = x_train[-val_size:]
y_val = y_train[-val_size:]
x_train = x_train[:-val_size]
y_train = y_train[:-val_size]

# Prosta eksploracja danych
plt.figure(figsize=(10, 2))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(x_train[i].squeeze(), cmap="gray")
    plt.title(str(y_train[i]))
    plt.axis("off")
plt.suptitle("Przykładowe obrazy ze zbioru treningowego")
plt.show()


## 2. Budowa modelu CNN

In [ ]:

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.summary()


## 3. Trening i zapis wag

In [ ]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train,
    epochs=7,
    batch_size=128,
    validation_data=(x_val, y_val)
)

# Zapis wyuczonych wag modelu
model.save('mnist_cnn_model.h5')


## 4. Ewaluacja i wizualizacja efektów klasyfikacji

In [ ]:

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

# Wizualizacja wyników na fragmencie testowym
y_pred = np.argmax(model.predict(x_test[:30]), axis=1)

plt.figure(figsize=(15, 4))
for i in range(30):
    plt.subplot(3, 10, i+1)
    plt.imshow(x_test[i].squeeze(), cmap="gray")
    color = "green" if y_pred[i] == y_test[i] else "red"
    plt.title(f"GT:{y_test[i]}
P:{y_pred[i]}", color=color)
    plt.axis("off")
plt.suptitle("Wyniki predykcji: zielony=poprawnie, czerwony=błędnie")
plt.show()


## 5. Predykcja na nieznanych obrazach

In [ ]:

# Przygotuj własny obraz (28x28, grayscale), tu demo na szumie
unknown_imgs = np.random.rand(3, 28, 28, 1)
unknown_preds = np.argmax(model.predict(unknown_imgs), axis=1)

plt.figure(figsize=(10, 2))
for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.imshow(unknown_imgs[i].squeeze(), cmap="gray")
    plt.title(f"Przewidziana klasa: {unknown_preds[i]}")
    plt.axis("off")
plt.suptitle("Predykcja na nieznanych obrazach (przykład/demo)")
plt.show()


## 6. Metryki: accuracy, confusion matrix

In [ ]:

y_test_pred = np.argmax(model.predict(x_test), axis=1)
acc = accuracy_score(y_test, y_test_pred)
cm = confusion_matrix(y_test, y_test_pred)
print(f"Accuracy (test): {acc:.4f}")

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Przewidziana klasa")
plt.ylabel("Prawdziwa klasa")
plt.title("Macierz pomyłek (Confusion Matrix)")
plt.show()


## 7. Wizualizacja feature maps (aktywacji) po pierwszej Conv2D

In [ ]:

activation_model = models.Model(inputs=model.input, outputs=model.layers[0].output)
activations = activation_model.predict(x_test[:1])  # pierwszy obraz testowy

plt.figure(figsize=(10, 4))
for i in range(activations.shape[-1]):
    plt.subplot(4, 8, i+1)
    plt.imshow(activations[0, :, :, i], cmap="viridis")
    plt.axis("off")
plt.suptitle("Feature maps po 1. Conv2D")
plt.show()
